In [2]:
import requests
from scrapy.http import TextResponse
import pandas as pd
import numpy as np
import time
from selenium import webdriver
import dbnomics
from dbnomics import fetch_series
import json
import quandl
import scrapy
from scrapy.crawler import CrawlerProcess

In [7]:
PATH = r"C:\Users\Roben\Desktop\Work Folder\NewLibrary\ChromeDriver\chromedriver.exe"
browser = webdriver.Chrome(PATH)
URL = 'http://rate.am/am/armenian-dram-exchange-rates/central-bank-armenia'
browser.get(URL)
time.sleep(2)
data = {}


exchange = ["USD", "EUR", "RUR","GBP"]
years = [2020,2019,2018]
for j in years:
    browser.find_element_by_css_selector("select[name='ctl00$Content$dlYear']").click()
    time.sleep(3)
    browser.find_element_by_css_selector(f"option[value='{j}']").click()
    time.sleep(3)
    for i in exchange:
        browser.find_element_by_css_selector("select[name='ctl00$Content$dlCurrency']").click()
        time.sleep(3)
        browser.find_element_by_css_selector(f"option[value={i}]").click()
        time.sleep(3)
        page = browser.page_source
        response = TextResponse(body=page,url=URL,encoding="utf-8")
        data.update({i+str(j):scrapRateAm(response,j,i)})
    

time.sleep(5)
browser.close()
   

In [6]:
def scrapRateAm(response,year,exchange):
    table = response.css("table[class='cb']")
    rows = table.css("tbody>tr")
    data = []
    for i in range(1,32):
        day = get_text(rows[i].css("td:nth-child(1)::text").extract())
        January = get_text(rows[i].css("td:nth-child(2)::text").extract())
        February = get_text(rows[i].css("td:nth-child(3)::text").extract())
        March = get_text(rows[i].css("td:nth-child(4)::text").extract())
        April = get_text(rows[i].css("td:nth-child(5)::text").extract())
        May = get_text(rows[i].css("td:nth-child(6)::text").extract())
        June = get_text(rows[i].css("td:nth-child(7)::text").extract())
        July = get_text(rows[i].css("td:nth-child(8)::text").extract())
        August = get_text(rows[i].css("td:nth-child(9)::text").extract())
        September = get_text(rows[i].css("td:nth-child(10)::text").extract())
        October = get_text(rows[i].css("td:nth-child(11)::text").extract())
        November = get_text(rows[i].css("td:nth-child(12)::text").extract())
        December = get_text(rows[i].css("td:nth-child(13)::text").extract())
        data.append({'Day':day,'January':January,'February':February,'March':March,'April':April,'May':May,'June':June,'July':July,'August':August,'September':September,'October':October,'November':November,'December':December})
   
    return pd.DataFrame(data)

In [8]:
def get_text(array):
    if len(array)==0:
        return ''
    else:
        if array[0].strip()=='X' or array[0].strip()=='':
            return ''
        return float(array[0].strip())

In [9]:
def getVariation(year,exchange):
    maxV = 0
    minV = 100000
    for label, content in data[f"{exchange + str(year)}"].iteritems():
        if label != 'Day':
             for i in content:
                if i!='':
                    if maxV<i:
                        maxV=i
                    if minV>i:
                        minV=i
    return maxV-minV
    



In [10]:
def monthly_difference(exchange):
    diff = []
    for i in data[f'{exchange}2019'].columns[1:]:
        diff.append(abs(data[f'{exchange}2019'][data[f'{exchange}2019'][i] != ''].mean()[i]-data[f'{exchange}2018'][data[f'{exchange}2018'][i] != ''].mean()[i]))
    return diff

In [11]:
# Problem 1 part a

getVariation(2019,"EUR")

41.23000000000002

In [12]:
# Problem 1 part b

diff = monthly_difference('EUR')
maxV = 0
minV = 1000
for i in diff:
    if(i<minV):
        minV = i
    if(i>maxV):
        maxV = i
finVar = maxV-minV
print(finVar)

27.231753846153993


Problem 2

In [17]:
data = quandl.get("WIKI/TSLA")

In [18]:
# Average daily percentage change of opening price.

data.pct_change().mean()['Open']

0.0019498764186026505

In [19]:
# Daily change of range between highest and lowest prices. 

data['Diff'] = data['High'] - data['Low']
data.pct_change().median()['Diff']

-0.016474576271185815

Problem 3

In [20]:
URL = "https://www.iban.com/country-codes"
page = requests.get(URL)
response = TextResponse(body=page.text,url=URL,encoding="utf-8")
codes = response.css(".table tr td:nth-child(2)::text").extract()

In [21]:
data = {}
for i in codes:
    df = fetch_series(f"IMF/DOT/A.AM.TMG_CIF_USD.{i.upper()}");
    if not df.empty:
        data[i] = df

Could not load series: {'dataset_code': 'DOT', 'message': 'Could not load series', 'provider_code': 'IMF', 'series_code': 'A.AM.TMG_CIF_USD.AX'}
Could not load series: {'dataset_code': 'DOT', 'message': 'Could not load series', 'provider_code': 'IMF', 'series_code': 'A.AM.TMG_CIF_USD.AD'}
Could not load series: {'dataset_code': 'DOT', 'message': 'Could not load series', 'provider_code': 'IMF', 'series_code': 'A.AM.TMG_CIF_USD.AQ'}
Could not load series: {'dataset_code': 'DOT', 'message': 'Could not load series', 'provider_code': 'IMF', 'series_code': 'A.AM.TMG_CIF_USD.AM'}
Could not load series: {'dataset_code': 'DOT', 'message': 'Could not load series', 'provider_code': 'IMF', 'series_code': 'A.AM.TMG_CIF_USD.AW'}
Could not load series: {'dataset_code': 'DOT', 'message': 'Could not load series', 'provider_code': 'IMF', 'series_code': 'A.AM.TMG_CIF_USD.BM'}
Could not load series: {'dataset_code': 'DOT', 'message': 'Could not load series', 'provider_code': 'IMF', 'series_code': 'A.AM.TM

In [22]:
years = {}
for i in data:
    for index, row in data[i].iterrows():
        year = row['original_period']
        if years.get(year) is None:
            years.update({year: row['value']})
        else:
            years[year] = years[year]+row['value']
max(years)

'2019'

In [28]:
dflast = []
for i in data:
    row = data[i][data[i]['original_period']=='2019']
    if not row.empty:
        dflast.append(row)
dflast = pd.concat(dflast)
dflast.sort_values('value',ascending=False).head()
# The 3rd largest import partner of Armenia is Germany

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,FREQ,REF_AREA,INDICATOR,COUNTERPART_AREA,Frequency,Reference Area,Indicator,Counterpart Reference Area
26,annual,IMF,DOT,Direction of Trade Statistics (DOTS),A.AM.TMG_CIF_USD.RU,"Annual – Armenia – Goods, Value of Imports, Co...",2019,2019-01-01,1478.544383,1478.544383,A,AM,TMG_CIF_USD,RU,Annual,Armenia,"Goods, Value of Imports, Cost, Insurance, Frei...",Russian Federation
26,annual,IMF,DOT,Direction of Trade Statistics (DOTS),A.AM.TMG_CIF_USD.CN,"Annual – Armenia – Goods, Value of Imports, Co...",2019,2019-01-01,751.114839,751.114839,A,AM,TMG_CIF_USD,CN,Annual,Armenia,"Goods, Value of Imports, Cost, Insurance, Frei...",China
27,annual,IMF,DOT,Direction of Trade Statistics (DOTS),A.AM.TMG_CIF_USD.DE,"Annual – Armenia – Goods, Value of Imports, Co...",2019,2019-01-01,381.680041,381.680041,A,AM,TMG_CIF_USD,DE,Annual,Armenia,"Goods, Value of Imports, Cost, Insurance, Frei...",Germany
26,annual,IMF,DOT,Direction of Trade Statistics (DOTS),A.AM.TMG_CIF_USD.IR,"Annual – Armenia – Goods, Value of Imports, Co...",2019,2019-01-01,324.928166,324.928166,A,AM,TMG_CIF_USD,IR,Annual,Armenia,"Goods, Value of Imports, Cost, Insurance, Frei...","Iran, Islamic Republic of"
27,annual,IMF,DOT,Direction of Trade Statistics (DOTS),A.AM.TMG_CIF_USD.US,"Annual – Armenia – Goods, Value of Imports, Co...",2019,2019-01-01,279.076573,279.076573,A,AM,TMG_CIF_USD,US,Annual,Armenia,"Goods, Value of Imports, Cost, Insurance, Frei...",United States


Problem 4

In [32]:
api_key="Provide your key tavari cav"

In [33]:
def show_distance(marzkentron1, marzkentron2):
    data = requests.get(f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={marzkentron1}+Armenia&destinations={marzkentron2}+Armenia&key={api_key}").json()
    distance = data['rows'][0]['elements'][0]['distance']['text']
    print(f"From {marzkentron1} to {marzkentron2} it is {distance}")

In [37]:
marzkentrons = ['Ashtarak', 'Artashat', 'Armavir', 'Gavar','Gyumri', 'Hrazdan','Ijevan','Kapan','Vanadzor','Yeghegnadzor']
for m1 in range(len(marzkentrons)):
    for m2 in range(m1+1,len(marzkentrons)):
        show_distance(marzkentrons[m1],marzkentrons[m2])

From Ashtarak to Artashat it is 51.9 km
From Ashtarak to Armavir it is 39.6 km
From Ashtarak to Gavar it is 104 km
From Ashtarak to Gyumri it is 98.4 km
From Ashtarak to Hrazdan it is 64.2 km
From Ashtarak to Ijevan it is 143 km
From Ashtarak to Kapan it is 320 km
From Ashtarak to Vanadzor it is 98.6 km
From Ashtarak to Yeghegnadzor it is 143 km
From Artashat to Armavir it is 63.2 km
From Artashat to Gavar it is 123 km
From Artashat to Gyumri it is 152 km
From Artashat to Hrazdan it is 83.6 km
From Artashat to Ijevan it is 162 km
From Artashat to Kapan it is 269 km
From Artashat to Vanadzor it is 148 km
From Artashat to Yeghegnadzor it is 92.1 km
From Armavir to Gavar it is 138 km
From Armavir to Gyumri it is 97.0 km
From Armavir to Hrazdan it is 97.8 km
From Armavir to Ijevan it is 176 km
From Armavir to Kapan it is 329 km
From Armavir to Vanadzor it is 138 km
From Armavir to Yeghegnadzor it is 153 km
From Gavar to Gyumri it is 168 km
From Gavar to Hrazdan it is 49.2 km
From Gavar to 

Problem 5

In [3]:
class MenuAmSpider(scrapy.Spider):
    name = 'menu'
    start_urls = ['https://www.menu.am/home.html?status=all&sort=default']

    custom_settings = {
        "FEED_FORMAT":"json",
        "FEED_URI":"menu.json",
    }
    
    def parse(self, response):
        for item in response.css('.item'):
            rating=0 
            if len(item.css('.new_rates_block ::text').extract())>0:
                rating = float(item.css('.new_rates_block ::text').extract()[0])
            yield {"title": item.css('.title ::attr(title)').extract()[0],
                  "hours": item.css('.new_list_time_block_inner ::text').extract()[0],
                   "rating": rating,
                   "category": item.css('.restType ::text').extract()[0] ,
                   "link": "https://www.menu.am/" +item.css('.title ::attr(href)').extract()[0] 
                  }



In [4]:
scraper = CrawlerProcess()
scraper.crawl(MenuAmSpider)
scraper.start()

2020-07-31 15:57:40 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: scrapybot)
2020-07-31 15:57:40 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.18362-SP0
2020-07-31 15:57:40 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-31 15:57:40 [scrapy.crawler] INFO: Overridden settings:
{}
2020-07-31 15:57:40 [scrapy.extensions.telnet] INFO: Telnet Password: 360e524de6c107a1
2020-07-31 15:57:40 [py.warnings] WARNING: C:\Users\Roben\Anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:210: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)

2020-07-31 

2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Օլիմպ Սպորտ Սնունդ', 'hours': '13:00 - 20:00', 'rating': 0, 'category': 'Գաստրոնոմ', 'link': 'https://www.menu.am//restaurant/olimp-sport.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Էլի ս Լահմաջուն', 'hours': '10:00 - 20:00', 'rating': 4.6, 'category': 'Սիրիական', 'link': 'https://www.menu.am//restaurant/elie-s-lahmajun.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Պիցցա Է Վինո', 'hours': '10:00 - 23:00', 'rating': 0, 'category': 'Եվրոպական, Իտալական', 'link': 'https://www.menu.am//restaurant/pizza-e-vino.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Տաք Կճուճ Ֆուդ Կորտ', 'hou

2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Քոֆի Հաուս', 'hours': '09:00 - 20:00', 'rating': 0, 'category': 'Սուրճ և Թեյ', 'link': 'https://www.menu.am//restaurant/coffee-house.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Լիմոնե', 'hours': '11:00 - 23:30', 'rating': 2.7, 'category': 'Իտալական', 'link': 'https://www.menu.am//restaurant/limone.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Կաֆե Եգեր', 'hours': '11:00 - 00:00', 'rating': 0, 'category': 'Գերմանական', 'link': 'https://www.menu.am//restaurant/cafe-jager.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Դե լա Ռոզ', 'hours': '10:00 - 00:00', 'rating': 3.0, 'category'

2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Սուշի Մուշի', 'hours': '24 ժամ', 'rating': 4.1, 'category': 'Ասիական, Ճապոնական', 'link': 'https://www.menu.am//restaurant/sushi-mushi.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Պասիֆլորա', 'hours': '24 ժամ', 'rating': 3.3, 'category': 'Ծաղիկներ և Փուչիկներ', 'link': 'https://www.menu.am//restaurant/pasion-flower.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Փարթիտտո', 'hours': '09:30 - 21:00', 'rating': 4.9, 'category': 'Ծաղիկներ և Փուչիկներ', 'link': 'https://www.menu.am//restaurant/partytto.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Արև Յոգա Նվեր Քարտ', 'hours': '10:30 

2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Anahit.am', 'hours': '24 ժամ', 'rating': 3.4, 'category': 'Ծաղիկներ և Փուչիկներ', 'link': 'https://www.menu.am//restaurant/anahitam.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Փարթի Թայմ', 'hours': '10:00 - 18:00', 'rating': 5.0, 'category': 'Ծաղիկներ և Փուչիկներ', 'link': 'https://www.menu.am//restaurant/party-time.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Սեգաֆրեդո', 'hours': '10:00 - 20:00', 'rating': 5.0, 'category': 'Եվրոպական, Սուրճ և Թեյ, Իտալական', 'link': 'https://www.menu.am//restaurant/segafredo-zanetti.html'}
2020-07-31 15:57:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Արտ Լանչ', 'h

...

2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Պուշկինի Ղարս', 'hours': '10:00 - 22:00', 'rating': 5.0, 'category': 'Արևելյան', 'link': 'https://www.menu.am//restaurant/pushkini-ghars.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Քամանչա Ռեստորան', 'hours': '10:00 - 23:00', 'rating': 4.0, 'category': 'Հայկական', 'link': 'https://www.menu.am//restaurant/qamancha-restaurant.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Վայն Ռեփաբլիկ', 'hours': '12:00 - 21:30', 'rating': 5.0, 'category': 'Եվրոպական, Թայլանդական', 'link': 'https://www.menu.am//restaurant/wine-republic.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'ՍԿԱՏ Բիր Հաուս'

2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Ինդիան Մեհակ Ռեստորան', 'hours': '13:00 - 22:30', 'rating': 4.2, 'category': 'Ասիական, Հնդկական', 'link': 'https://www.menu.am//restaurant/indian-mehak-restaurant.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Թոմի Հիլֆիգեր', 'hours': '10:30 - 21:30', 'rating': 5.0, 'category': 'ՆՎԵՐ ՔԱՐՏԵՐ', 'link': 'https://www.menu.am//restaurant/tommy-hilfiger-gift-card.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Մինուռ', 'hours': '09:00 - 22:00', 'rating': 0, 'category': 'Եվրոպական', 'link': 'https://www.menu.am//restaurant/minour.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Հարվի', 'hour

2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Յամ-Յամ Դոնաթս Աբովյան', 'hours': '11:00 - 20:00', 'rating': 4.5, 'category': 'Քաղցրավենիք, Ամերիկյան', 'link': 'https://www.menu.am//restaurant/yum-yum-donuts-yerevan.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Օստ Բիստրո', 'hours': '10:30 - 21:30', 'rating': 4.2, 'category': 'Արևելյան, Հայկական', 'link': 'https://www.menu.am//restaurant/ost-bistro-mashtots.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Սաբթայթլ', 'hours': '10:00 - 23:59', 'rating': 4.6, 'category': 'Ամերիկյան, Բուրգեր և Սենդվիչ', 'link': 'https://www.menu.am//restaurant/subtitle.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=defa

2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': '21 Սուշի Քլաբ', 'hours': '24 ժամ', 'rating': 4.2, 'category': 'Ճապոնական', 'link': 'https://www.menu.am//restaurant/21-sushi-club.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Թոշկա', 'hours': '11:00 - 18:00', 'rating': 3.0, 'category': 'Սիրիական', 'link': 'https://www.menu.am//restaurant/toshka.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Բուկինիստ Գրախանութների Ցանց', 'hours': '10:00 - 21:00', 'rating': 0, 'category': 'ՆՎԵՐ ՔԱՐՏԵՐ', 'link': 'https://www.menu.am//restaurant/bookinist-bookstores-chain-gift-card.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Սերվիս Լենդ', 'hours'

2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Հայաթ Շաուրմա', 'hours': '11:00 - 21:00', 'rating': 5.0, 'category': 'Արաբական', 'link': 'https://www.menu.am//restaurant/hayat-shaurma.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Նաիրիան Նվեր Քարտ', 'hours': '10:00 - 20:00', 'rating': 0, 'category': 'ՆՎԵՐ ՔԱՐՏԵՐ', 'link': 'https://www.menu.am//restaurant/nairian-gift-card.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Մաթևոսյան Վայն', 'hours': '10:00 - 17:00', 'rating': 5.0, 'category': 'Ըմպելիքներ և Խորտիկներ', 'link': 'https://www.menu.am//restaurant/matevosyan-wine.html'}
2020-07-31 15:57:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.menu.am/home.html?status=all&sort=default>
{'title': 'Հեյզ', 'hours

In [5]:
# The categories wuth highest rating

df = pd.read_json('menu.json')
df.groupby('category').mean().sort_values('rating', ascending=False).head()

,rating
category,
"Արտադրված է Հայաստանում, Գրքեր, Եվրոպական",5.0
Հացաբուլկեղեն,5.0
"Եվրոպական, Սուրճ և Թեյ, Իտալական",5.0
Արաբական,5.0
"Միրգ և բանջարեղեն, Հայկական",5.0


In [8]:
# Number of restaurants that close before 19:00

restaurants = 0
hour = 19
for index, row in df.iterrows():
    closingTime = row['hours'][8:]
    closingTimeHour = closingTime.strip()[:2]
    closingTimeMinute = closingTime.strip()[3:]
    if len(closingTimeHour)>0:
        if int(closingTimeHour) < hour and  int(closingTimeHour) > 8: 
            restaurants = restaurants + 1
        elif int(closingTimeHour) == hour:
            if int(closingTimeMinute) == 0:
                restaurants = restaurants + 1
print(restaurants)

51
